In [1]:
path = "../runs_RegretNet/X.npy"
path_ADV = "../runs_RegretNet/ADV_550.npy"
import numpy as np
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
ADV = np.load(path_ADV)
ADV.shape


(1, 640000, 5, 3)

In [15]:
dataset = np.load(path)

In [16]:
dataset.shape

(640000, 5, 3)

In [17]:
import sys
sys.path.append("../")
!ls ..
from core import base
import core.base.base_generator as base_generator
import json
import easydict
config = json.load(open("../runs_RegretNet/setting_default_config/seed_0/config.json", "r"))
config = easydict.EasyDict(config)
print(config)

core  layers   main.py	   plots      requirements.txt	runs_RegretNet	utility
data  LICENSE  mechanisms  README.md  run_configs	target_nets	utils
{'regret_type': 'standard', 'architecture': 'RegretNet', 'distribution_type': 'uniform_01', 'min': 2, 'max': 3, 'num_agents': 5, 'num_items': 3, 'save_data': 'runs_RegretNet/setting_default_config/seed_0', 'dir_name': 'runs_RegretNet', 'plot': {'bool': False, 'n_points': 201}, 'distill': {'architecture': None, 'validate_target_misreports': True, 'train_misreports': True}, 'net': {'init': 'gu', 'activation': 'tanh', 'num_a_layers': 3, 'num_p_layers': 3, 'num_p_hidden_units': 20, 'num_a_hidden_units': 20, 'layer_norm': False, 'hid_att': 16, 'hid': 32, 'n_attention_layers': 1, 'n_attention_heads': 2, 'activation_att': 'tanh', 'pos_enc': False, 'pos_enc_part': 1, 'pos_enc_item': 1, 'n_exch_layers': 3, 'hid_exch': 32, 'activation_exch': 'relu'}, 'train': {'seed': 42, 'restore_iter': 0, 'max_iter': 750, 'learning_rate': 0.001, 'data': 'fixed', 'num_

In [18]:
from __future__ import absolute_import, division, print_function

import numpy as np

from core.base.base_generator import BaseGenerator


class Generator(BaseGenerator):
    def __init__(self, config, mode, X=None, ADV=None):
        super(Generator, self).__init__(config, mode)
        self.build_generator(X=X, ADV=ADV)

    def save_data(self, iter):
        return super().save_data(iter)

    def generate_random_X(self, shape):
        return np.random.rand(*shape)

    def generate_random_ADV(self, shape):
        return np.random.rand(*shape)


In [19]:
dataset = Generator(config, "train")

In [20]:
X, ADV, perm = next(dataset.gen_online())
print(X.shape, ADV.shape, perm)

x = torch.from_numpy(X).float().to(device)

(512, 5, 3) (1, 512, 5, 3) None


# GAN

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter  


class Discriminator(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.disc = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.LeakyReLU(0.01),
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.disc(x)


class Generator(nn.Module):
    def __init__(self, z_dim, img_dim):
        super().__init__()
        self.gen = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.LeakyReLU(0.01),
            nn.Linear(256, img_dim),
            nn.Tanh(),  
        )

    def forward(self, x):
        return self.gen(x)


device = "cuda" if torch.cuda.is_available() else "cpu"
lr = 3e-4
z_dim = 64
image_dim = X.shape[1]*X.shape[2]
batch_size = 32
num_epochs = 50

disc = Discriminator(image_dim).to(device)
gen = Generator(z_dim, image_dim).to(device)
fixed_noise = torch.randn((batch_size, z_dim)).to(device)
transforms = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)

opt_disc = optim.Adam(disc.parameters(), lr=lr)
opt_gen = optim.Adam(gen.parameters(), lr=lr)
criterion = nn.BCELoss()
writer_fake = SummaryWriter(f"logs/fake")
writer_real = SummaryWriter(f"logs/real")
step = 0

for epoch in range(num_epochs):
    X, ADV, perm = next(dataset.gen_online())
    x = torch.from_numpy(X).float().to(device)
    real = x.view(-1, X.shape[1]*X.shape[2]).to(device)
    batch_size = real.shape[0]

    noise = torch.randn(batch_size, z_dim).to(device)
    fake = gen(noise)
    disc_real = disc(real).view(-1)
    lossD_real = criterion(disc_real, torch.ones_like(disc_real))
    disc_fake = disc(fake).view(-1)
    lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
    lossD = (lossD_real + lossD_fake) / 2
    disc.zero_grad()
    lossD.backward(retain_graph=True)
    opt_disc.step()

    output = disc(fake).view(-1)
    lossG = criterion(output, torch.ones_like(output))
    gen.zero_grad()
    lossG.backward()
    opt_gen.step()

    if epoch % 1 == 0:
        print(
            f"Epoch [{epoch}/{num_epochs}] \
                  Loss D: {lossD:.4f}, loss G: {lossG:.4f}"
        )

        with torch.no_grad():
            fake = gen(fixed_noise).reshape(-1, 1, X.shape[1], X.shape[2])
            data = real.reshape(-1, 1, X.shape[1], X.shape[2])
            img_grid_fake = torchvision.utils.make_grid(fake, normalize=True)
            img_grid_real = torchvision.utils.make_grid(data, normalize=True)

            writer_fake.add_image(
                "Mnist Fake Images", img_grid_fake, global_step=step
            )
            writer_real.add_image(
                "Mnist Real Images", img_grid_real, global_step=step
            )
            step += 1


Epoch [0/50]                   Loss D: 0.6843, loss G: 0.6703
Epoch [1/50]                   Loss D: 0.6840, loss G: 0.6651
Epoch [2/50]                   Loss D: 0.6828, loss G: 0.6630
Epoch [3/50]                   Loss D: 0.6828, loss G: 0.6592
Epoch [4/50]                   Loss D: 0.6828, loss G: 0.6547
Epoch [5/50]                   Loss D: 0.6825, loss G: 0.6510
Epoch [6/50]                   Loss D: 0.6809, loss G: 0.6492
Epoch [7/50]                   Loss D: 0.6824, loss G: 0.6436
Epoch [8/50]                   Loss D: 0.6826, loss G: 0.6402
Epoch [9/50]                   Loss D: 0.6834, loss G: 0.6355
Epoch [10/50]                   Loss D: 0.6826, loss G: 0.6317
Epoch [11/50]                   Loss D: 0.6843, loss G: 0.6269
Epoch [12/50]                   Loss D: 0.6848, loss G: 0.6219
Epoch [13/50]                   Loss D: 0.6843, loss G: 0.6191
Epoch [14/50]                   Loss D: 0.6860, loss G: 0.6144
Epoch [15/50]                   Loss D: 0.6855, loss G: 0.6106
Ep